In [24]:
from ibl_pipeline.analyses import behavior as behavior_analyses
from ibl_pipeline import behavior as behavior
from ibl_pipeline import subject, acquisition
from ibl_pipeline.utils import psychofit as psy
import plotting_utils as putils
from uuid import UUID
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

In [25]:
subj_uuid = (subject.Subject & 'subject_nickname="IBL_13"').fetch1('subject_uuid')
key = {'subject_uuid': subj_uuid,
   'session_date': datetime.date(2019, 4, 8)}
sessions = behavior_analyses.BehavioralSummaryByDate.PsychResults * behavior_analyses.BehavioralSummaryByDate.ReactionTimeContrast & key

data_mean = []
data_error = []

for session in sessions.fetch('KEY'):
    contrasts, prob_left, reaction_time, ci_low, ci_high = \
        (sessions & session).fetch1(
            'signed_contrasts', 'prob_left', 'reaction_time_contrast', 
            'reaction_time_ci_low', 'reaction_time_ci_high')
    error_low = reaction_time - ci_low
    error_high = ci_high - reaction_time
    
    curve_color, error_color = putils.get_color(prob_left, 0.3)

    rt_data = go.Scatter(
        x=contrasts.tolist(),
        y=reaction_time.tolist(),
        marker=dict(
            size=6,
            color=curve_color),
        mode='markers+lines',
        name=f'p_left = {prob_left}'
    )

    rt_errorbar = go.Scatter(
        x=contrasts.tolist(),
        y=reaction_time.tolist(),
        error_y=dict(
            type='data',
            array=error_high.tolist(),
            arrayminus=error_low.tolist(),
            visible=True,
            color=error_color
        ),
        marker=dict(
            size=6,
        ),
        mode='none',
        showlegend=False
    )
    
    data_mean.append(rt_data)
    data_error.append(rt_errorbar)

layout = go.Layout(
    width=630,
    height=400,
    title='Reaction time - contrast', 
    xaxis={'title':'Contrast (%)'}, 
    yaxis={'title':'Reaction time (s)'},
)

data = data_error
for element in data_mean:
    data.append(element)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

In [22]:
f = open("date_reaction_time_contrast.json","w")
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()